# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re

from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end - start} \n')
        return result

    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        res_args = tuple(args)
        res_kwargs = {k:v for k,v in kwargs}
        print(f'Функция вызвана с параметрами: \n {res_args}, {res_kwargs} \n')
        return result

    return wrapper

## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    cnt = 0
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        nonlocal cnt
        cnt += 1
        if cnt%10 in [2,3,4] and cnt not in [12,13,14]:
            print(f'Функция была вызвана: {cnt} раза \n')
        else:
            print(f'Функция была вызвана: {cnt} раз \n')
        return result

    return wrapper

## Задание 4

In [6]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}

  def fmemo(*args):
    if args in fmemo.cache:
        return fmemo.cache[args]

    result = func(*args)
    fmemo.cache[args] = result
    return res

  fmemo.cache = cache
  return fmemo

## Тестирование

In [7]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз "

print(word_count('whole'))

Время выполнения функции word_count: 0.4997958110000127 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 1 раз 

Cлово whole встречается 176 раз 


In [8]:
# проверяем счетчик запусков функции
for _ in range(4):
    print(word_count('whole'))
    print('----------------------------------')

Время выполнения функции word_count: 0.43583517899998014 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 2 раза 

Cлово whole встречается 176 раз 
----------------------------------
Время выполнения функции word_count: 0.4149234909999677 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 3 раза 

Cлово whole встречается 176 раз 
----------------------------------
Время выполнения функции word_count: 0.4142330319999701 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 4 раза 

Cлово whole встречается 176 раз 
----------------------------------
Время выполнения функции word_count: 0.42652524499999345 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 5 раз 

Cлово whole встречается 176 раз 
----------------------------------


In [9]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [10]:
# измеряем время выполнения
res = benchmark(fib)
res(35)
res(35)

Время выполнения функции fib: 3.85497811700003 

Время выполнения функции fib: 3.742703980999977 



9227465

In [15]:
@memo
def fib2(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [16]:
# измеряем время выполнения
res2 = benchmark(fib2)
res2(35)
res2(35)
# при первом вызове результат выполнения функции записывается в кэш
# при последующих запусках результат берется из кэша и время на вычисление не требуется

Время выполнения функции fmemo: 3.7857166480000046 

Время выполнения функции fmemo: 2.983999991101882e-06 



9227465